# Convert prediction results to labels(将预测结果转为标签)

In [1]:
import os
import pandas as pd
import numpy as np

import math
import random

from pandas import set_option

In [21]:
# mpl.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
set_option("display.max_rows", 15)
set_option('display.width', 400)
np.set_printoptions(suppress=True, threshold=5000)

In [3]:
# 使用GPU训练时候，打开下面注释
os.environ['CUDA_VISIBLE_DEVICES']='0'

# Preparation work (data processing, result saving location definition)(准备工作,数据处理，结果保存位置定义)

## Input prediction result dataset location(输入结果数据集位置)

In [17]:
usual_logs_DataPath = 'facies_csv_results/bigru-self-atten_test/'

In [18]:
filename_usual_logs = 'YX-58井_facies_vectors_0.1_7_Pred_R_04271055.csv'   # 青山口组；  泉头组
usual_logs_DataPath = os.path.join(usual_logs_DataPath,filename_usual_logs)
print(usual_logs_DataPath)

facies_csv_results/bigru-self-atten_test/YX-58井_facies_vectors_0.1_7_Pred_R_04271055.csv


In [37]:
well_no = "YX-58"

## Storage location of lithofacies dataset（岩相数据集保存位置）

In [6]:
facies_logs_DataPath = 'facies_csv_results/'

In [7]:
method = os.path.dirname(usual_logs_DataPath).split("/")[-1]

In [8]:
filename_facies_logs =  'facies_pred_' + method + '.csv'

In [9]:
filename_facies_logs

'facies_pred_bigru-self-atten_test.csv'

In [10]:
facies_logs_DataPath = os.path.join(facies_logs_DataPath,filename_facies_logs)

In [117]:
facies_logs_DataPath

'facies_csv_results/facies_pred_bigru-self-atten_test.csv'

# Lithographic data conversion（岩相数据标定)

## ## Read input curves and expert lithofacies classification data（读取输入曲线和专家岩相分类数据)

In [26]:
input_vector_data = pd.read_csv(usual_logs_DataPath,engine='python',encoding='GBK',sep = ',')

In [27]:
input_vector_data

,DEPTH,GR,CNL,DT,DEN,MSFL,RS,RD,Prediction,Facies
0,2013.3,141.5902,22.3832,98.4706,2.4714,0.6528,0.6723,0.6273,2,1
1,2013.4,138.6362,20.9666,95.5322,2.4768,0.6187,0.6664,0.6202,2,1
2,2013.5,133.8810,19.0250,91.1610,2.4840,0.5940,0.6772,0.6301,2,1
3,2013.6,127.3738,17.7330,86.7578,2.4968,0.6701,0.7177,0.6745,2,1
4,2013.7,120.9938,16.9672,83.8078,2.5114,0.8415,0.7759,0.7408,2,1
...,...,...,...,...,...,...,...,...,...,...
1077,2121.0,138.9920,19.5630,91.9960,2.5790,0.8238,0.7333,0.6824,1,3
1078,2121.1,133.5352,19.7814,89.8688,2.5622,0.7972,0.7630,0.7131,1,6
1079,2121.2,126.5028,19.7904,87.2976,2.5448,0.8582,0.8096,0.7632,1,6
1080,2121.3,120.2804,19.4012,84.6108,2.5292,1.0068,0.8679,0.8274,2,6


In [28]:
# 针对输入曲线
depth_cols = 'DEPTH'
# 针对岩相曲线
pred_col = 'Prediction'
depth_begin_col = '顶深'
depth_end_col = '底深'
facies_col = '岩性'
well_name_col = '井号'

In [29]:
resolution = 50
resolution_ratio = 1 / resolution

In [32]:
input_vector_data[depth_cols][9],len(input_vector_data)

(2014.2, 1082)

In [61]:
begin_depth_list = []
end_depth_list = []
pred_list = []
current_row_id = 0
cursor_facies_id = input_vector_data[pred_col][0]
cursor_depth = input_vector_data[depth_cols][0]
for depth in range(len(input_vector_data)):
    curent_depth = input_vector_data[depth_cols][depth]
    current_facies = input_vector_data[pred_col][depth]
    if cursor_facies_id !=  input_vector_data[pred_col][depth]:
        begin_depth_list.append(cursor_depth)
        end_depth_list.append(curent_depth)
        pred_list.append(cursor_facies_id)
        cursor_facies_id = current_facies
        cursor_depth = curent_depth
    else:
        pass

In [63]:
len(begin_depth_list),len(end_depth_list),len(pred_list)

(47, 47, 47)

In [35]:
facies_vector = input_vector_data

In [56]:
if well_name_col in facies_vector.columns:
    pd_well_no = np.full(len(input_vector_data),facies_vector[well_name_col][0])
else:
    pd_well_no = np.full(len(input_vector_data),well_no)

In [57]:
pd_well_no

array(['YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58', 'YX-58',
       'YX-58', 'YX-

## 岩相类别符号转换

In [44]:
# facies_label = np.unique(facies_vector[facies_col])
'''
facies_label = [
    "High organic matter layered shale lithofacies",
    "High organic striated layered shale lithofacies",
    "Medium organic striated layered shale lithofacies",
    "Low organic matter striated layered shale lithofacies",
    "Medium to low organic matter massive dolomite lithofacies",
    "Low organic matter massive shell limestone lithofacies"
    "Low organic matter massive siltstone lithofacies"
]
'''
facies_labels = ['高有机质层状页岩相', '高有机质纹层状页岩相','中有机质纹层状页岩相','低有机质纹层状页岩相',
                '中低有机质块状白云岩相', '低有机质块状介壳灰岩相', '低有机质块状粉砂岩相']

In [45]:
# init varible
depth_min = 2300
depth_max = 2400
math.floor(depth_min),math.ceil(depth_max)

(2300, 2400)

In [46]:
# for i in range(math.floor(depth_min),math.ceil(depth_max)+1):
#     print(i)
len(facies_labels[0])

9

In [47]:
label_max_lenth = 0

In [48]:
for i in range(len(facies_labels)):
    if label_max_lenth < len(facies_labels[i]):
        label_max_lenth = len(facies_labels[i])

In [49]:
len(input_vector_data)

1082

In [50]:
def generate_random_str(randomlength):
    """
    生成一个指定长度的随机字符串
    """
    random_str = ''
    base_str = 'ABCDEFGHIGKLMNOPQRSTUVWXYZabcdefghigklmnopqrstuvwxyz0123456789'
    length = len(base_str) - 1
    for i in range(randomlength):
        random_str += base_str[random.randint(0, length)]
    return random_str
# ————————————————
# 版权声明：本文为CSDN博主「geerniya」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/geerniya/java/article/details/78497765

In [51]:
fill_label = generate_random_str(label_max_lenth)
fill_label

'HHFMkQLUxNF'

In [84]:
facies_list = np.full(len(pred_list),"no_label")

In [85]:
len(begin_depth_list),len(end_depth_list),len(pred_list)

(47, 47, 47)

In [86]:

lower_depth_index = 0
# 深度列标尺初始化
# current_depth = input_vector_data[depth_cols][0]
facies_label_m_pred = 0
real_class_num = len(facies_labels)
for i in range(len(pred_list)):
    depth_min = begin_depth_list[i]
    depth_max = end_depth_list[i]
    current_facies_label = pred_list[i]
    if facies_label_m_pred < 0 or facies_label_m_pred >= real_class_num:
        current_facies_id = "unpredictable"
    else:
        current_facies_id = facies_labels[current_facies_label]
        # current_well_no = facies_vector[well_name_col][i]
        print("depth_min,depth_max:",depth_min,depth_max)
        facies_list[i] = current_facies_id
        print("No_" + str(i) + "layers" + ": " + facies_list[i])

print("Label Work Finished!")

depth_min,depth_max: 2013.3 2014.2
No_0layers: 中有机质纹层状页
depth_min,depth_max: 2014.2 2014.7
No_1layers: 低有机质块状粉砂
depth_min,depth_max: 2014.7 2015.6
No_2layers: 高有机质纹层状页
depth_min,depth_max: 2015.6 2016.4
No_3layers: 中有机质纹层状页
depth_min,depth_max: 2016.4 2017.6
No_4layers: 低有机质纹层状页
depth_min,depth_max: 2017.6 2027.7
No_5layers: 中有机质纹层状页
depth_min,depth_max: 2027.7 2032.8
No_6layers: 高有机质纹层状页
depth_min,depth_max: 2032.8 2033.4
No_7layers: 低有机质块状粉砂
depth_min,depth_max: 2033.4 2034.3
No_8layers: 高有机质纹层状页
depth_min,depth_max: 2034.3 2034.9
No_9layers: 中有机质纹层状页
depth_min,depth_max: 2034.9 2036.1
No_10layers: 高有机质纹层状页
depth_min,depth_max: 2036.1 2037.2
No_11layers: 中有机质纹层状页
depth_min,depth_max: 2037.2 2039.1
No_12layers: 高有机质纹层状页
depth_min,depth_max: 2039.1 2040.0
No_13layers: 低有机质块状粉砂
depth_min,depth_max: 2040.0 2040.9
No_14layers: 高有机质纹层状页
depth_min,depth_max: 2040.9 2042.0
No_15layers: 中有机质纹层状页
depth_min,depth_max: 2042.0 2044.2
No_16layers: 高有机质纹层状页
depth_min,depth_max: 2044.2 2044.7
No_17l

In [87]:
# facies_list len(begin_depth_list),len(end_depth_list)

In [88]:
# wellname_colunm_data = pd.DataFrame(pd_well_no, columns=["Well Name"])
begin_depth_colunm_data = pd.DataFrame(begin_depth_list, columns=["begin depth"])
end_depth_colunm_data = pd.DataFrame(end_depth_list, columns=["end depth"])

In [89]:
facies_colunm_data = pd.DataFrame(facies_list, columns=["Facies"])

In [90]:
facies_colunm_data

,Facies
0,中有机质纹层状页
1,低有机质块状粉砂
2,高有机质纹层状页
3,中有机质纹层状页
4,低有机质纹层状页
...,...
42,高有机质纹层状页
43,高有机质层状页岩
44,高有机质纹层状页
45,高有机质层状页岩


In [91]:
# wellname_colunm_data
len(facies_colunm_data)

47

In [92]:
pd_data = pd.concat([begin_depth_colunm_data, end_depth_colunm_data ,facies_colunm_data],axis=1)

In [93]:
pd_data

,begin depth,end depth,Facies
0,2013.3,2014.2,中有机质纹层状页
1,2014.2,2014.7,低有机质块状粉砂
2,2014.7,2015.6,高有机质纹层状页
3,2015.6,2016.4,中有机质纹层状页
4,2016.4,2017.6,低有机质纹层状页
...,...,...,...
42,2094.1,2101.0,高有机质纹层状页
43,2101.0,2103.0,高有机质层状页岩
44,2103.0,2110.9,高有机质纹层状页
45,2110.9,2111.6,高有机质层状页岩


In [94]:
save_no_label_pred_info = False
if save_no_label_pred_info == True:
    data_save = pd_data[pd_data["Facies"] != "no_label"].reset_index(drop=True)
else:
    data_save = pd_data

In [95]:
data_save

,begin depth,end depth,Facies
0,2013.3,2014.2,中有机质纹层状页
1,2014.2,2014.7,低有机质块状粉砂
2,2014.7,2015.6,高有机质纹层状页
3,2015.6,2016.4,中有机质纹层状页
4,2016.4,2017.6,低有机质纹层状页
...,...,...,...
42,2094.1,2101.0,高有机质纹层状页
43,2101.0,2103.0,高有机质层状页岩
44,2103.0,2110.9,高有机质纹层状页
45,2110.9,2111.6,高有机质层状页岩


# Mark Results Save(标记结果保存)

## Result file header information(结果文件文件头信息)

In [111]:
# fianl_facies_logs = well_name + '_'+ str(begin_depth) + '-'+ str(end_depth) +'_facies_vectors_' + str(1/resolution)+ '.csv' 
fianl_facies_logs = well_no + '_facies_vectors_pred.csv'

In [112]:
print(fianl_facies_logs)

YX-58_facies_vectors_pred.csv


## Result dataset save location(结果数据集保存位置)

In [113]:
Using_DataPath = 'facies_data/train/'
if not os.path.exists(Using_DataPath):
    os.makedirs(Using_DataPath)

In [114]:
# Using_DataPath = os.path.join(Using_DataPath,fianl_facies_logs)

In [115]:
resolution_ratio = 1 / resolution
resolution_ratio

0.1

## The saved result curve(保存的结果曲线)

In [96]:
data_save

,begin depth,end depth,Facies
0,2013.3,2014.2,中有机质纹层状页
1,2014.2,2014.7,低有机质块状粉砂
2,2014.7,2015.6,高有机质纹层状页
3,2015.6,2016.4,中有机质纹层状页
4,2016.4,2017.6,低有机质纹层状页
...,...,...,...
42,2094.1,2101.0,高有机质纹层状页
43,2101.0,2103.0,高有机质层状页岩
44,2103.0,2110.9,高有机质纹层状页
45,2110.9,2111.6,高有机质层状页岩


In [119]:
# 输出txt
# data_save.to_csv(os.path.join(Using_DataPath,fianl_facies_logs),mode='w',float_format='%.4f',sep='\t',index=None,header=True)
data_save.to_csv(os.path.join(Using_DataPath,fianl_facies_logs),mode='w',float_format='%.4f',encoding='GBK',index=None,header=True)
print("Algorithm is Finished!!")

Algorithm is Finished!!


In [ ]:
# if paly_music == True:
#     mixer.init()
#     music_path = "music/"
#     file = "MISIA - 星のように.mp3"
#     music_file = os.path.join(music_path,file)
#     mixer.music.load(music_file)
#     mixer.music.play()
#     time.sleep(6)
#     mixer.music.stop()
#         # exit()